# IMERG netcdf to a single file 

In [8]:
import numpy as np 
import pandas as pd 
import netCDF4
from datetime import datetime as Time
from datetime import timedelta as Delta
import h5py
from tqdm import tqdm 

def nc_to_pd(nc):
    try:
        Keys = list(nc.keys())
    except:
        Keys = list(nc.variables)
    Panda = {}
    for k in Keys:
        try:
            Panda[k]= np.array(nc[k] )
            PD = pd.DataFrame( Panda )
        except:
            Panda[k] = np.array(nc[k]).T[0]
            PD = pd.DataFrame( Panda )
            pass
    return PD , Panda

def number_to_date(number):
    start_date  = "01/01/1900"
    date_1 = pd.to_datetime(start_date)
    end_date = date_1 + pd.DateOffset(hours=number) 
    return end_date


def Dict_to_hdf(Dict , of ):
    try: HDF.close()   
    except: pass
    
    HDF = h5py.File(of, 'w' , driver='core')
    n_k = len(Dict)
    Keys = list(Dict.keys())

    for i in  tqdm( range( 0 , n_k), desc="Writing HDF5..." , ncols= 100  ,colour='red') :
        k = Keys[i] 
        test = HDF.create_dataset( k , data = Dict[k]  )
    HDF.close()   

def dd(d) :
    if d< 10 :return '0' + str(d)
    else: return  str(d)

### Inputs

In [11]:
Folder_imerg = r'd:\DatasetAUS\IMERG-LD6/'
of  = r'd:\DatasetAUS\IMERG_HQP_20130101-20240323-1.hdf5'

start = Time( 2013  , 1  , 1 )
end = Time( 2024 , 3  , 23 )




In [10]:

n_days = (end - start).days

out_dict = {}

for d in  tqdm( range( 0 , n_days+1), desc="Reading IMERG files ..." , ncols= 100  ,colour='blue') :
    this_time = start +  Delta(days= d)
    file =  '3B-DAY-L.MS.MRG.3IMERG.' + dd(this_time.year) +dd(this_time.month)  + dd(this_time.day) + '-S000000-E235959.V06.nc4.nc4'
    fp = Folder_imerg + file
    nc = netCDF4.Dataset(fp)
    out_dict[dd(this_time.year)+ '-'+dd(this_time.month) + '-' + dd(this_time.day)] = np.array(nc['HQprecipitation'] )[0]
    # cal_p = np.array(nc['precipitationCal'] )[0]

out_dict['lon'] = np.array(nc['lon'] )
out_dict['lat'] = np.array(nc['lat'] )

Dict_to_hdf(out_dict , of  )



Reading IMERG files ...: 100%|██████████████████████████████████| 4100/4100 [00:58<00:00, 69.68it/s]


OSError: Unable to synchronously create file (unable to truncate a file which is already open)

In [12]:
Dict_to_hdf(out_dict , of  )


Writing HDF5...: 100%|██████████████████████████████████████████| 4102/4102 [27:29<00:00,  2.49it/s]


In [ ]:
def Join_IMERG(Folder_imerg  , start , end , band ):
    bands ={
    'HQprecipitation' ,
    'precipitationCal' }
    
    n_days = (end - start).days

    out_dict = {}

    for d in  tqdm( range( 0 , n_days+1), desc="Reading IMERG files ..." , ncols= 100  ,colour='blue') :
        this_time = start +  Delta(days= d)
        file =  '3B-DAY-L.MS.MRG.3IMERG.' + dd(this_time.year) +dd(this_time.month)  + dd(this_time.day) + '-S000000-E235959.V06.nc4.nc4'
        fp = Folder_imerg + file
        nc = netCDF4.Dataset(fp)
        out_dict[dd(this_time.year)+ '-'+dd(this_time.month) + '-' + dd(this_time.day)] = np.array(nc[band] )[0]
        # cal_p = np.array(nc['precipitationCal'] )[0]

    out_dict['lon'] = np.array(nc['lon'] )
    out_dict['lat'] = np.array(nc['lat'] )

    return out_dict
            